1\. Write a function that converts number representation (bin<->dec<->hex)

In [2]:
def int_to_bin(x):
    if type(x) != int: print("Sorry, %f is not a an integer, i can't convert it into it's binary representation" %x)
    else:
        x_bin=[]
        while x != 0:
            x_bin.append(x%2)
            x = int(x/2)
        x_bin.reverse()
        str_x_bin = ''.join(str(e) for e in x_bin) 
    return str_x_bin


In [14]:
def bin_to_int(x):
    if type(x) != str: print("Sorry, %s is not a an string, i can't convert it into it's decimal representation" %x)
    else:
        x_dec = 0
        for j in range(len(x)):
            x_dec += int(x[j]) * 2**(len(x) - (j + 1))
    return x_dec

In [21]:
def hex_to_int(x):
    conversioni = {"0":0,"1":1,"2":2,"3":3,"4":4,"5":5,"6":6,"7":7,"8":8,"9":9,"a":10,"b":11,"c":12,"d":13,"e":14,"f":15}
    if type(x) != str: print("Sorry, %s is not a an string, i can't convert it into it's decimal representation" %x)
    else:
        x_dec = 0
        for j in range(len(x)):
            x_dec +=conversioni[x[j]] * 16**(len(x) - (j + 1))
    return x_dec        


In [30]:
def int_to_hex(x):
    conversioni = {0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9', 10: 'a', 11: 'b', 12: 'c', 13: 'd', 14: 'e', 15: 'f'}
    if type(x) != int: print("Sorry, %f is not a an integer, i can't convert it into it's hexadecimal representation" %x)
    else:
        x_bin=[]
        while x != 0:
            x_bin.append(conversioni[x%16])
            x = int(x/16)
        x_bin.reverse()
        str_x_bin = ''.join(str(e) for e in x_bin) 
    return str_x_bin


In [32]:
def bin_to_hex(x):
    x_hex = []
    if type(x) != str: print("Sorry, %s is not a an string, i can't convert it into it's hexadecimal representation" %x)
    else:
        temp_list=list(x)
        temp_list.reverse()
        x = ''.join(str(e) for e in temp_list)
        a = 0
        b = 0
        while len(x) > b:
            b = b + min(len(x) - b, 4)
            temp = x[a:b]
            temp_list=list(temp)
            temp_list.reverse()
            temp = ''.join(str(e) for e in temp_list)
            x_hex.append((int_to_hex(bin_to_int(temp))))
            a +=4
        x_hex.reverse()
        str_x_hex = ''.join(str(e) for e in x_hex)   
    return str_x_hex   


In [35]:
def hex_to_bin(x):
    if type(x) != str: print("Sorry, %s is not a an string, i can't convert it into it's binary representation" %x)
    else: 
        return(int_to_bin(hex_to_int(x)))
x="3cd"
print(hex_to_int(x))

973


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 
